#### Notebook for Demand Analysis of the Painkiller Market

In [3]:
# Importing Libraries
import pandas as pd
import numpy as np
from math import ceil
import seaborn as sns
from linearmodels.iv import compare
from collections import OrderedDict
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from linearmodels.iv import IV2SLS, IVGMM
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()

# Set up Linear Model
lr = LinearRegression()


#### Functions 

In [4]:
# Function for returning the week of a month, needed for data loading
def week_of_month(dt):
    """ Returns the week of the month for the specified date.
    """

    first_day = pd.to_datetime(dt.replace(day=1))

    dom = dt.day
    adjusted_dom = dom + first_day.weekday()

    return int(ceil(adjusted_dom/7.0))


#### Data Loading and Cleaning

In [5]:
# Setting Filepath
filepath = '../data/'

# Reading in RDS file
readRDS = robjects.r['readRDS']
df = readRDS(filepath + 'analgesics.RDS')
data = pandas2ri.ri2py(df)

#Removing rows where OK column is 0
data = data[data.ok==1].reset_index(drop=True)

# Removing rows where quantities are below 0.01, these weeks have nonsensical pricing data
data = data[(data.quant_weekly>0.01)]

# Adding constant for regression
data['const'] = 1

# Converting UPC data type
data.upc = data.upc.astype(int).astype(str)

# Removing Excederin from the data, due to the fact that its Aspirin + Paracetamol, not suitable for Nested Logit
data = data[data.upc!='31981000166']

# Matching Unique UPCs to descriptions
upc_map=[]
# Getting descriptions and sizes
for upc in data.upc.unique():
    des = data.loc[data.upc==upc,'descrip'].iloc[0]
    size = data.loc[data.upc==upc,'nsize'].iloc[0]
    upc_map.append([upc,des, size])

# Full product names from description and size
products = [upc_map[i][1] + '_' + str(upc_map[i][2]) for i in range(0, len(upc_map))]

# Assigning groups by active ingredients, generic vs branded and sizes
for upc, desc, size in upc_map:
    if ('TYLENOL' in desc) | ('NON-ASP' in desc):
        data.loc[data.upc==upc,'Nest']=1
    elif ('ADVIL' in desc) | ('IBUPROFEN' in desc):
        data.loc[data.upc==upc,'Nest']=2
    else:
        data.loc[data.upc==upc,'Nest']=3
    #generics    
    if ('DOM' in desc):
        data.loc[data.upc==upc,'Generic']=1
    else:
        data.loc[data.upc==upc,'Generic']=0
    #sizes
    if (size==24) or (size==30):
        data.loc[data.upc==upc,'size_group']=1
    elif (size==50) or (size==60):
        data.loc[data.upc==upc,'size_group']=2
    else:
        data.loc[data.upc==upc,'size_group']=3

# Compute market share numbers

# Weekly sales by UPC, week and store 
data.loc[:, 'Product_sales'] = data.groupby(['upc', 'week', 'store']).quant_weekly.transform(sum)

# Weekly Sales per store
data.loc[:, 'Weekly_sales'] = data.groupby(['week', 'store']).quant_weekly.transform(sum)

# Market share of a given product as defined by Sales / Mean Customer Count
data.loc[:, 'Market_share'] = data.Product_sales / data.msize_meancustcount_weekly

# Outside Share
data.loc[:, 'Outside_share'] = (data.msize_meancustcount_weekly - data.Weekly_sales)/ data.msize_meancustcount_weekly

# Calculating Log difference between market share and outside share for estimation
data.loc[:, 'log_diff'] = np.log(data.Market_share) - np.log(data.Outside_share)

# Compute group market share numbers -by ingredient
data.loc[:, 'Ingroup_sales'] = data.groupby(['upc', 'week', 'store', 'Nest']).quant_weekly.transform(sum)
data.loc[:, 'Group_sales'] = data.groupby(['week', 'store','Nest']).quant_weekly.transform(sum)
data.loc[:, 'Group_share_ing'] = np.log(data.Ingroup_sales / data.Group_sales)

# Compute group market share numbers - by generic vs branded
data.loc[:, 'Ingroup_sales_Gen'] = data.groupby(['upc', 'week', 'store', 'Generic']).quant_weekly.transform(sum)
data.loc[:, 'Group_sales_Gen'] = data.groupby(['week', 'store','Generic']).quant_weekly.transform(sum)
data.loc[:, 'Group_share_Gen'] = np.log(data.Ingroup_sales_Gen / data.Group_sales_Gen)

/Users/17867029/anaconda3/lib/python3.6/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


In [6]:
# Extracting exogenous variables

#Customer Demographics
demos = data.iloc[:,129:172].copy()

#Sales Data
sales = data.loc[:, ['salesum_b_help', 'sale_b_weekly','salesum_c_help', 'sale_c_weekly', 
                     'salesum_g_help','sale_g_weekly', 'salesum_s_help', 'sale_s_weekly',
                     'sale_all_weekly']]

#Chosen Control variables
use_cols = ['age9', 'income', 'ethnic', 'hsizeavg', 'nocar', 'hvalmean', 
            'retired', 'unemp', 'educ', 'sale_all_weekly']

Loading and preparing exchange rate data for Instrumental Variables

In [10]:

# Load historical currency rates
# USD vs CNY
CNY = pd.read_csv(filepath + 'USD_CNY_Historical_Data.csv')

# USD vs INR
INR = pd.read_csv(filepath + 'USD_INR_Historical_Data.csv')

# EUR vs USD
EUR = pd.read_csv(filepath + 'EUR_USD_Historical_Data.csv')
# Inverse for correct specification
EUR.Price = 1/ EUR.Price

# Extract datetime variables
CNY.Date = pd.to_datetime(CNY.Date)
INR.Date = pd.to_datetime(INR.Date)
EUR.Date = pd.to_datetime(EUR.Date)
CNY['Month'] = CNY.Date.dt.month
CNY['Year'] = CNY.Date.dt.year
CNY['week_in_mon'] = CNY.Date.apply(week_of_month)
INR['Month'] = INR.Date.dt.month
INR['Year'] = INR.Date.dt.year
INR['week_in_mon'] = INR.Date.apply(week_of_month)
EUR['Month'] = EUR.Date.dt.month
EUR['Year'] = EUR.Date.dt.year
EUR['week_in_mon'] = EUR.Date.apply(week_of_month)

# Transforming the original data to fit the dates
data.date = pd.to_datetime(data.date)
data['Month'] = data.date.dt.month
data['Year'] = data.date.dt.year

#Merge FX rates in data set
gb = CNY.groupby(['Year', 'Month', 'week_in_mon']).Price.median().reset_index()
gb.columns = ['Year', 'Month', 'week_in_mon', 'CNY_Px']
data = data.merge(gb, on=['Year', 'Month', 'week_in_mon'], how='left')
gb = INR.groupby(['Year', 'Month', 'week_in_mon']).Price.median().reset_index()
gb.columns = ['Year', 'Month', 'week_in_mon', 'INR_Px']
data = data.merge(gb, on=['Year', 'Month', 'week_in_mon'], how='left')
gb = EUR.groupby(['Year', 'Month', 'week_in_mon']).Price.median().reset_index()
gb.columns = ['Year', 'Month', 'week_in_mon', 'EUR_Px']
data = data.merge(gb, on=['Year', 'Month', 'week_in_mon'], how='left')

#### Predicting wholesaleprices to be used in the IV-Estimation

In [13]:
dummies = pd.concat((pd.get_dummies(data.size_group, prefix='Size_'),pd.get_dummies(data.firm_id, prefix='Firm_'),
                    pd.get_dummies(data.Nest, prefix='Ingred_')), axis=1)


In [14]:
# Make dummies to predict wholesaleprices
dummies = pd.concat((pd.get_dummies(data.size_group, prefix='Size_'),pd.get_dummies(data.firm_id, prefix='Firm_'),
                    pd.get_dummies(data.Nest, prefix='Ingred_')), axis=1)

# Dataframe with exogenous variables for wholesaleprices
data_iv = data.loc[:, ['CNY_Px', 'INR_Px', 'EUR_Px', 'whprice_weekly' ,'const']]
data_iv.fillna(method='ffill', inplace=True)


# Extract wholesaleprices 
target = data_iv.whprice_weekly

# Drop Wholesaleprices from previous dataframe
data_iv.drop('whprice_weekly', axis=1, inplace=True)


# Drop exchange rates
#data_iv.drop(['CNY_Px', 'INR_Px', 'EUR_Px'],axis=1, inplace=True)

# Predict Wholesaleprices from Exchange rates
reg_whPx = sm.OLS(endog=target, exog=data_iv, missing='drop', hasconst=True)

# Model results
results = reg_whPx.fit()
print(results.summary())

# Store predicted values in dataframe
data['pred_whprice'] = results.predict(data_iv)

                            OLS Regression Results                            
Dep. Variable:         whprice_weekly   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     122.2
Date:                Fri, 16 Nov 2018   Prob (F-statistic):           4.48e-79
Time:                        01:43:11   Log-Likelihood:             7.4754e+05
No. Observations:              344796   AIC:                        -1.495e+06
Df Residuals:                  344792   BIC:                        -1.495e+06
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
CNY_Px        -0.0006   4.43e-05    -12.610      0.0

In [15]:
data_iv.head(5)

,CNY_Px,INR_Px,EUR_Px,const
0,4.7339,17.999,0.733676,1
1,4.7339,17.999,0.733676,1
2,4.7339,17.999,0.733676,1
3,4.7339,17.999,0.733676,1
4,4.7339,17.999,0.733676,1


#### Multinomial Logit Model

In [16]:
# Setting up variables to be used as instruments 

# Competitors sales
data['Comp_sales_all'] = data.groupby(['week', 'store']).sale_all_weekly.transform('sum') - data.sale_all_weekly

# Competitor group sales
data['Comp_gpsls_all'] = data.groupby(['week', 'store', 'Nest']).sale_all_weekly.transform('sum') - data.sale_all_weekly

# Competitor size 
data['Comp_gpsls_size_all'] = data.groupby(['week', 'store', 'size_group']).sale_all_weekly.transform('sum') - data.sale_all_weekly

# Competitor generic 
data['Comp_gpsls_gen_all'] = data.groupby(['week', 'store', 'Generic']).sale_all_weekly.transform('sum') - data.sale_all_weekly

In [17]:
# Set-up categoricals for dummies
data['firm_id'] = data['firm_id'].astype('category')
data['size_group'] = data['size_group'].astype('category')
data['Nest'] = data['Nest'].astype('category')

#Multinomial logit with demographics
ml_demo = IV2SLS(dependent=data['log_diff'],
            exog=data.loc[:,use_cols + ['const']],
            endog=data['price_weekly'],
            instruments=data['pred_whprice']).fit(cov_type='clustered', clusters=data.store)

In [18]:
results = OrderedDict()
#results['ML_No Exog'] = ml_noexo
results['ML + Demos'] = ml_demo
print(compare(results))
alpha = -ml_demo.params['price_weekly']

          Model Comparison         
                         ML + Demos
-----------------------------------
Dep. Variable              log_diff
Estimator                   IV-2SLS
No. Observations             344796
Cov. Est.                 clustered
R-squared                   -17.627
Adj. R-squared              -17.627
F-statistic                  1144.9
P-value (F-stat)             0.0000
==================      ===========
age9                         2.1261
                           (0.6807)
income                      -0.2715
                          (-0.7546)
ethnic                      -0.7216
                          (-3.0899)
hsizeavg                    -0.0790
                          (-0.2807)
nocar                       -0.0572
                          (-0.0949)
hvalmean                     0.0013
                           (0.7897)
retired                     -0.9129
                          (-0.9359)
unemp                       -0.4181
                          (-

In [36]:
%%time

# Calculate elasticities
# Takes about 8min
n_upcs = len(data.upc.unique())
elastic_iv = np.zeros((n_upcs, n_upcs))
week_gp = data.groupby(['week','upc'])['price_weekly', 'Market_share'].median().reset_index()

for i in range(0, n_upcs):
    upc1 = data.upc.unique()[i]
    
    for j in range(i, n_upcs):
        
        upc2 = data.upc.unique()[j]
        tmp=[]
        tmp2=[]
        for w in range(0, len(data.week.unique())):
            week = data.week.unique()[w]
            try:
                px1 = week_gp.loc[(week_gp.week==week) & (week_gp.upc == upc1),'price_weekly'].values[0]
                ms1 = week_gp.loc[(week_gp.week==week) & (week_gp.upc == upc1),'Market_share'].values[0]        
                px2 = week_gp.loc[(week_gp.week==week) & (week_gp.upc == upc2),'price_weekly'].values[0]
                ms2 = week_gp.loc[(week_gp.week==week) & (week_gp.upc == upc2),'Market_share'].values[0]
                if i==j:
                    # Own Price Elastictiy
                    tmp.append(-alpha*(1-ms1)*px1)
                else:
                    # Cross Price Elasticity
                    tmp.append(alpha*px2*ms2)
            except:
                pass
            
        if i==j:
            elastic_iv[i,j] = np.nanmedian(tmp)
        else:
            elastic_iv[i,j] = np.nanmedian(tmp)
            elastic_iv[j,i] = np.nanmedian(tmp)

CPU times: user 8min 16s, sys: 40.6 ms, total: 8min 16s
Wall time: 8min 16s


In [37]:
# Generating Elasticities matrix
products = [upc_map[i][1] + '_' + str(upc_map[i][2]) for i in range(0, len(upc_map))]

# Make Dataframe
elastic_iv_df = pd.DataFrame(elastic_iv, columns=products, index=products)
own_elastic_iv = list(zip(products, np.round(np.diag(elastic_iv_df),2)))

# Extract Own Pric 
own_df = pd.DataFrame(own_elastic_iv, columns = ['Product', 'Own Px Elasticity'])
own_df.set_index('Product').sort_values('Own Px Elasticity').to_csv(filepath+'ML_own')

In [41]:
own_df

,Product,Own Px Elasticity
0,DOM COATED ASPIRIN_100,-0.26
1,DOM X/S NON-ASPIRIN_100,-0.47
2,DOM E/S NON-ASP CPLT_50,-0.53
3,DOM IBUPROFEN_100,-0.58
4,DOM IBUPROFEN_50,-0.72
5,TYLENOL X/S CAPLET_24,-1.60
6,TYLENOL X/S CAPLET_50,-1.15
7,TYLENOL X/S TABLETS_100,-0.66
8,TYLENOL X/S GELCAPS_100,-0.87
9,TYLENOL X/S GELCAPS_24,-1.62


#### Nested Logit Model

In [55]:
# Group by branded/generic within ingredient groups
# Compute group market share numbers
data.loc[:, 'Ingroup_sales_L2'] = data.groupby(['upc', 'week', 'store', 'Generic', 'Nest']).quant_weekly.transform(sum)
data.loc[:, 'Group_sales_L2'] = data.groupby(['week', 'store','Generic', 'Nest']).quant_weekly.transform(sum)
data.loc[:, 'Group_share_gen_ing'] = np.log(data.Ingroup_sales_L2 / data.Group_sales_L2)
data['Comp_gpsls_L2_all'] = data.groupby(['week', 'store', 'Generic', 'Nest']).sale_all_weekly.transform('sum') - data.sale_all_weekly

In [56]:
# Nested logit on Ingredients
iv_nest_ing = IV2SLS(dependent=data['log_diff'],
            exog=data.loc[:,use_cols +['const']],
            endog=data.loc[:,['price_weekly', 'Group_share_ing']],
            instruments=data.loc[:,['pred_whprice','Comp_gpsls_all']]).fit(cov_type='clustered', clusters=data.store)

# Nested logit on Generic v Branded
iv_nest_gen = IV2SLS(dependent=data['log_diff'],
            exog=data.loc[:,use_cols +['const']],
            endog=data.loc[:,['price_weekly', 'Group_share_Gen']],
            instruments=data.loc[:,['pred_whprice','Comp_gpsls_gen_all']]).fit(cov_type='clustered', clusters=data.store)


#Nested logit on double nest L2: Generic, L1: Ingredient
iv_nest_ing_gen = IV2SLS(dependent=data['log_diff'],
            exog=data.loc[:,use_cols +['const']],
            endog=data.loc[:,['price_weekly', 'Group_share_Gen', 'Group_share_gen_ing']],
            instruments=data.loc[:,['pred_whprice','Comp_gpsls_gen_all', 'Comp_gpsls_L2_all']]).fit(cov_type='clustered', clusters=data.store)

# Extracting coefficients of the Nested Logit Model on Ingredients
alpha = - iv_nest_ing.params['price_weekly']
sigma = iv_nest_ing.params['Group_share_ing']

In [45]:
# Results Table

resultsn = OrderedDict()
resultsn['ML '] = ml_demo
resultsn['Nest Gen.'] = iv_nest_gen
resultsn['Nest Ing.'] = iv_nest_ing
resultsn['Nest L1:Gen, L2:Ing'] = iv_nest_ing_gen
#resultsn['Nest FE'] = iv_nest_fixed
cmp = compare(resultsn)
cmp

,ML,Nest Gen.,Nest Ing.,"Nest L1:Gen, L2:Ing"
Dep. Variable,log_diff,log_diff,log_diff,log_diff
Estimator,IV-2SLS,IV-2SLS,IV-2SLS,IV-2SLS
No. Observations,344796,344796,344796,344796
Cov. Est.,clustered,clustered,clustered,clustered
R-squared,0.3328,0.5950,0.6209,0.5364
Adj. R-squared,0.3328,0.5950,0.6209,0.5364
F-statistic,3276.8,4771.8,9833.7,5459.5
P-value (F-stat),0.0000,0.0000,0.0000,0.0000
=====================,===========,===========,===========,===========
age9,3.8289,4.0242,4.2789,4.0891


In [52]:
%%time

# Compute elasticities for nested logit
# Takes about 4 min

upc_df = pd.DataFrame(upc_map, columns=['upc', 'Name', 'Size'])
week_gp = data.groupby(['week','upc'])['price_weekly', 'Market_share', 
                                           'Group_share_ing'].median().reset_index()

weeks = len(data.week.unique())

nests = data.Nest.unique()

for nest in nests:
    nest_upcs = data[data.Nest==nest].upc.unique()
    elastic_x = np.zeros((len(nest_upcs), len(nest_upcs)))
    products=[]
    for u in nest_upcs:
        products.append(str(upc_df.loc[upc_df.upc==u,'Name'].iloc[0]) + ' ' +\
                        str(upc_df.loc[upc_df.upc==u,'Size'].iloc[0]))
    for i in range(0, len(nest_upcs)):
        upc1 = nest_upcs[i]
        g1 = nest
        for j in range(i, len(nest_upcs)):
            #print(i,j)
            upc2 = nest_upcs[j]
            g2 = nest
            tmp=[]
            tmp2=[]
            for w in range(0, weeks):
                week = data.week.unique()[w]
                try:
                    px1 = week_gp.loc[(week_gp.week==week) & (week_gp.upc == upc1),'price_weekly'].values[0]
                    ms1 = week_gp.loc[(week_gp.week==week) & (week_gp.upc == upc1),'Market_share'].values[0]
                    gs1 = week_gp.loc[(week_gp.week==week) & (week_gp.upc == upc1),'Group_share_ing'].values[0]            
                    px2 = week_gp.loc[(week_gp.week==week) & (week_gp.upc == upc2),'price_weekly'].values[0]
                    ms2 = week_gp.loc[(week_gp.week==week) & (week_gp.upc == upc2),'Market_share'].values[0]
                    gs2 = week_gp.loc[(week_gp.week==week) & (week_gp.upc == upc2),'Group_share_ing'].values[0]
                    if i==j:
                        tmp.append(-alpha*px1*(1-sigma*gs1-(1-sigma)*ms1)/(1-sigma))
                    elif g1==g2:
                        tmp.append(-alpha*px2*(sigma*gs1+(1-sigma)*ms2)/(1-sigma))
                except:
                    pass

            if i==j:
                elastic_x[i,j] = np.nanmedian(tmp)
            else:
                elastic_x[i,j] = np.nanmedian(tmp)
    
    # Elasticities matrix
    if nest==3:
        elastic_Asp = pd.DataFrame(elastic_x, columns=products, index=products)
    elif nest==1:
        elastic_Ace = pd.DataFrame(elastic_x, columns=products, index=products)
    else:
        elastic_Ibu = pd.DataFrame(elastic_x, columns=products, index=products)

CPU times: user 4min 45s, sys: 190 ms, total: 4min 45s
Wall time: 4min 45s


In [54]:
#Write to file

(np.round(elastic_Asp,2)).to_csv(filepath+'Aspirin Nest')
(np.round(elastic_Ace,2)).to_csv(filepath+'Acet Nest')
(np.round(elastic_Ibu,2)).to_csv(filepath+'Ibu Nest')

#### End of Document